<h1 align=center><font size = 5>Analisis Eleccion 2019 - Montevideo</font></h1>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/neovalle/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.2                |   py37h89c1867_0         3.0 MB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following packages will be UPDATED:

  conda                                4.8.5-py37hc8dfbb8_1 --> 4.9.2-py37h89c1867_0
  geopy                                  2.0.0-pyh9f0ad1d_0 --> 2.1.0-pyhd3deb0d_0



conda-4.9.2          | 3.0 MB    | ##################################### | 100% 
geopy-2.1.0          | 64 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Exe

<a id='itemA'></a>
## 1. Data Explore

In [2]:
dfs = pd.read_excel('../VotosMontevideo.xlsx')

In [3]:
print(dfs.head())
dfs.shape

   Circuito SERIE Municipio  Habilitados  No_Obs  Obs  Emitidos  EnBlanco  \
0         1   AAA         B          389     302    8       310         4   
1         2   AAA         B          389     344    4       348         7   
2         3   AAA         B          389     333    2       335         8   
3         4   AAA         B          389     317    2       319         4   
4         5   AAA         B          389     338    2       340         1   

   Anulados  PorSi   FA  PN  PC  PI  AP  PT  PE  PG  PV  PD  CA Desde  \
0         3      4  117  87  38   5   2   0   5   4   2   0  31     1   
1         8      3  129  81  23   6   4   0   6   2   2   0  73  6092   
2        10      1  124  67  35   5   4   1   3   2   5   0  68  7224   
3        10      2  133  62  20   5   3   0   3   6   1   0  68  8036   
4         5      3  164  78  26   4   5   0   2   1   5   1  43  9789   

     Hasta                                        Local  \
0   6091.0                 Centro SUB O

(2642, 26)

In [4]:
# Municipio is Z is not a physical localation - remove id
dfs = dfs[dfs.Municipio != 'Z']
dfs.shape

(2615, 26)

In [5]:
# get votes by PC vs Opposition (FA + PN + PI + AP + PT + PE + PG + PV + PD + CA)
municipio_votes=dfs.groupby(['Municipio'])['FA','PN','PC','PI','AP','PT','PE','PG','PV','PD','CA'].agg('sum').reset_index()
municipio_votes

/home/neovalle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,Municipio,FA,PN,PC,PI,AP,PT,PE,PG,PV,PD,CA
0,A,84472,25658,7204,1364,1744,134,2047,2068,1274,286,9792
1,B,49126,28264,14607,1829,1624,95,2167,961,1450,485,8620
2,C,55013,27337,13171,1701,1617,107,2432,1177,1491,456,8960
3,D,58034,26336,7980,1322,1124,89,1678,2051,1089,297,13757
4,E,43811,28034,17490,1589,1129,89,2191,986,1363,438,7017
5,F,53004,24749,6681,1229,979,54,1427,2125,1025,307,12040
6,G,52538,20349,7171,1211,1221,89,1735,1481,1078,282,7723
7,X,40247,32422,23507,2088,1348,54,2156,810,1238,494,6567


In [6]:
municipio_votes['FA_Percentage']=((municipio_votes['FA']/municipio_votes.iloc[:,0:12].sum(axis=1))*100).round(2)
municipio_votes['PN_Percentage']=((municipio_votes['PN']/municipio_votes.iloc[:,0:12].sum(axis=1))*100).round(2)
municipio_votes['PC_Percentage']=((municipio_votes['PC']/municipio_votes.iloc[:,0:12].sum(axis=1))*100).round(2)
municipio_votes['CA_Percentage']=((municipio_votes['CA']/municipio_votes.iloc[:,0:12].sum(axis=1))*100).round(2)
municipio_votes['Otros_Percentage']=(((municipio_votes['PI'] 
                                     +municipio_votes['AP']
                                     +municipio_votes['PT']
                                     +municipio_votes['PE']
                                     +municipio_votes['PG']
                                     +municipio_votes['PV']
                                     +municipio_votes['PD'])                                  
                                     /municipio_votes.iloc[:,0:12].sum(axis=1))*100).round(2)


In [7]:
municipio_votes['Total_Votes']=municipio_votes.iloc[:,0:12].sum(axis=1)
municipio_votes

,Municipio,FA,PN,PC,PI,AP,PT,PE,PG,PV,PD,CA,FA_Percentage,PN_Percentage,PC_Percentage,CA_Percentage,Otros_Percentage,Total_Votes
0,A,84472,25658,7204,1364,1744,134,2047,2068,1274,286,9792,62.09,18.86,5.30,7.20,6.55,136043
1,B,49126,28264,14607,1829,1624,95,2167,961,1450,485,8620,44.98,25.88,13.37,7.89,7.88,109228
2,C,55013,27337,13171,1701,1617,107,2432,1177,1491,456,8960,48.49,24.09,11.61,7.90,7.92,113462
3,D,58034,26336,7980,1322,1124,89,1678,2051,1089,297,13757,51.02,23.15,7.01,12.09,6.72,113757
4,E,43811,28034,17490,1589,1129,89,2191,986,1363,438,7017,42.07,26.92,16.80,6.74,7.48,104137
5,F,53004,24749,6681,1229,979,54,1427,2125,1025,307,12040,51.15,23.88,6.45,11.62,6.90,103620
6,G,52538,20349,7171,1211,1221,89,1735,1481,1078,282,7723,55.37,21.45,7.56,8.14,7.48,94878
7,X,40247,32422,23507,2088,1348,54,2156,810,1238,494,6567,36.28,29.23,21.19,5.92,7.38,110931


In [8]:
municipio_votes.to_excel('TotalesMontevideo.xlsx')

First interesting finding: there is a clear difference between municipalities, the extremes are Municipio A and Municipio X  
Government has 62% of approval in the former and only 36% in the latter.

In [9]:
# Import Municipios data
dfm = pd.read_excel('MunicipiosAddresses.xlsx')
dfm

,Municipio,Address,Latitude,Longitude
0,A,Carlos María Ramírez 862,-34.865730,-56.235827
1,B,Joaquín Requena 1701,-34.897304,-56.171140
2,C,L. A. de Herrera 4547,-34.867148,-56.171454
3,X,Brito del Pino 1590,-34.899690,-56.156650
4,D,Av. Gral. Flores 4694,-34.845525,-56.154466
5,E,Estadio Charrúa,-34.878506,-56.089357
6,F,Av. 8 de Octubre 4700,-34.858786,-56.133496
7,G,Cno. Castro 730,-34.859575,-56.213453


### First, visualise Montevideo and its municipalities

In [10]:
# get Mntevideo's Lat and Long
address = 'Montevideo, Uruguay'

geolocator = Nominatim(user_agent="montevideo")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of montevideo are {}, {}.'.format(latitude, longitude))

The geographical coordinate of montevideo are -34.9058916, -56.1913095.


In [11]:
# create map of montevideo using latitude and longitude values
map_montevideo = folium.Map(location=[latitude, longitude], zoom_start=12)
map_montevideo 

In [12]:
# add municipality centres to map

for lat, lng, municip in zip(dfm['Latitude'], dfm['Longitude'], dfm['Municipio']):
    label = '{}'.format(municip)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.1,
        parse_html=False).add_to(map_montevideo)  

map_montevideo

### First explore the extremes; find Categories in Municipalities A (index 0) and X (index 3)

#### Municipio A

In [13]:
# credentials
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 3000 # define radius
municipio_latitude=dfm.loc[0, 'Latitude']
municipio_longitude=dfm.loc[0,'Longitude']
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    municipio_latitude, 
    municipio_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
# Find categories
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print("Categories in Municipio A:")
nearby_venues.head()
nearby_venues

Categories in Municipio A:


/home/neovalle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Circulo De Tenis,Tennis Court,-34.862401,-56.205919
1,Parque del Prado,Garden,-34.859800,-56.205096
2,Campomar F5,Soccer Field,-34.864487,-56.215044
3,La fortaleza,Pizza Place,-34.874959,-56.250936
4,Rosedal,Garden,-34.859560,-56.205701
5,Parque José Nasazzi,Soccer Stadium,-34.861640,-56.208767
6,Estadio Belvedere,Soccer Field,-34.855107,-56.225360
7,Patio del Naranjo,Restaurant,-34.868393,-56.204319
8,Club Stockolmo,Basketball Court,-34.853753,-56.208681
9,Macro Mercado La Teja,Grocery Store,-34.861693,-56.231333


#### Municipio X

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 3000 # define radius
municipio_latitude=dfm.loc[3, 'Latitude']
municipio_longitude=dfm.loc[3,'Longitude']
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    municipio_latitude, 
    municipio_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [18]:
# Find categories
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print("Categories in Municipio X:")
nearby_venues.head()
nearby_venues

Categories in Municipio X:


/home/neovalle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,Bar Bass,Pub,-34.905184,-56.158088
1,Parque Batlle,Park,-34.898597,-56.156181
2,Ulpiano,Restaurant,-34.901611,-56.160266
3,Hey chopp,Bar,-34.901430,-56.168003
4,Tandory,Restaurant,-34.907692,-56.152395
5,La Esquina del Mundo,Café,-34.899524,-56.148022
6,ZonaTecno,Electronics Store,-34.898583,-56.167738
7,Sophilabs HQ,IT Services,-34.897425,-56.160791
8,Monumento a La carreta,Sculpture Garden,-34.892404,-56.155365
9,Obelisco a los Constituyentes de 1830,Monument / Landmark,-34.897667,-56.164751


#### Municipio B

In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 3000 # define radius
municipio_latitude=dfm.loc[1, 'Latitude']
municipio_longitude=dfm.loc[1,'Longitude']
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    municipio_latitude, 
    municipio_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [20]:
# Find categories
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print("Categories in Municipio B:")
nearby_venues.head()
nearby_venues

Categories in Municipio B:


/home/neovalle/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,name,categories,lat,lng
0,ZonaTecno,Electronics Store,-34.898583,-56.167738
1,Hey chopp,Bar,-34.901430,-56.168003
2,La Cigale,Ice Cream Shop,-34.899900,-56.169501
3,Mercado Ferrando,Food Court,-34.904439,-56.170653
4,Escarmuza Librería y Café,Bookstore,-34.906503,-56.171224
5,Obelisco a los Constituyentes de 1830,Monument / Landmark,-34.897667,-56.164751
6,Ganache,Café,-34.904224,-56.170715
7,Café Gourmand,French Restaurant,-34.906994,-56.175552
8,La Trastienda Club,Music Venue,-34.898455,-56.178493
9,Farmashop 55,Pharmacy,-34.901001,-56.173686


We have now found an interesting difference in categories between Municipio A and X: Municipio A has more sport and open spaces

### Explore municipalities

Since Montevideo has a 200sq km area, each of the 8 municipalities cover approx 25sq km that is a radius of approx 3000 meters

In [21]:
# function to find nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Municipio', 
                  'Municipio Latitude', 
                  'Municipio Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the function on each neighborhood and create a new dataframe called *municipio_venues*.

In [22]:
municipio_venues = getNearbyVenues(names=dfm['Municipio'],
                                   latitudes=dfm['Latitude'],
                                   longitudes=dfm['Longitude']
                                  )

A
B
C
X
D
E
F
G


In [23]:
municipio_venues.to_excel('MunicipioLugares.xlsx')
municipio_venues

,Municipio,Municipio Latitude,Municipio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,A,-34.865730,-56.235827,Circulo De Tenis,-34.862401,-56.205919,Tennis Court
1,A,-34.865730,-56.235827,Parque del Prado,-34.859800,-56.205096,Garden
2,A,-34.865730,-56.235827,Campomar F5,-34.864487,-56.215044,Soccer Field
3,A,-34.865730,-56.235827,La fortaleza,-34.874959,-56.250936,Pizza Place
4,A,-34.865730,-56.235827,Rosedal,-34.859560,-56.205701,Garden
5,A,-34.865730,-56.235827,Parque José Nasazzi,-34.861640,-56.208767,Soccer Stadium
6,A,-34.865730,-56.235827,Estadio Belvedere,-34.855107,-56.225360,Soccer Field
7,A,-34.865730,-56.235827,Patio del Naranjo,-34.868393,-56.204319,Restaurant
8,A,-34.865730,-56.235827,Club Stockolmo,-34.853753,-56.208681,Basketball Court
9,A,-34.865730,-56.235827,Macro Mercado La Teja,-34.861693,-56.231333,Grocery Store


In [24]:
# Check how many venues were returned for each Municipio

municipio_venues.groupby('Municipio').count()

,Municipio Latitude,Municipio Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Municipio,,,,,,
A,15,15,15,15,15,15
B,100,100,100,100,100,100
C,68,68,68,68,68,68
D,31,31,31,31,31,31
E,80,80,80,80,80,80
F,30,30,30,30,30,30
G,62,62,62,62,62,62
X,100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [25]:
print('There are {} uniques categories.'.format(len(municipio_venues['Venue Category'].unique())))

There are 110 uniques categories.


<a id='item3'></a>

## 3. Analyse Each Municipio

In [26]:
# one hot encoding
municipio_onehot = pd.get_dummies(municipio_venues[['Venue Category']], prefix="", prefix_sep="")

# add municipio column back to dataframe
municipio_onehot['Municipio'] = municipio_venues['Municipio'] 

# move municipio column to the first column
fixed_columns = [municipio_onehot.columns[-1]] + list(municipio_onehot.columns[:-1])
municipio_onehot = municipio_onehot[fixed_columns]

municipio_onehot.head()

,Municipio,American Restaurant,Art Museum,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Bookstore,Brewery,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Fountain,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,Hostel,Hot Dog Joint,Hotel,Housing Development,IT Services,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Latin American Restaurant,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pub,Research Laboratory,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Waterfront,Women's Store
0,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
municipio_onehot.shape

(486, 111)

#### group rows by municipio and by taking the mean of the frequency of occurrence of each category

In [28]:
municipio_grouped = municipio_onehot.groupby('Municipio').mean().reset_index()
municipio_grouped.to_excel('FrecuenciaLugaresPorMunicipio.xlsx')
municipio_grouped

,Municipio,American Restaurant,Art Museum,Athletics & Sports,BBQ Joint,Bakery,Bar,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Bookstore,Brewery,Burger Joint,Bus Station,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Donut Shop,Electronics Store,Fast Food Restaurant,Flea Market,Food,Food & Drink Shop,Food Court,Fountain,French Restaurant,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Historic Site,Hostel,Hot Dog Joint,Hotel,Housing Development,IT Services,Ice Cream Shop,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Kebab Restaurant,Latin American Restaurant,Market,Mediterranean Restaurant,Middle Eastern Restaurant,Monument / Landmark,Movie Theater,Multiplex,Museum,Music Venue,Nightclub,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pub,Research Laboratory,Restaurant,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Waterfront,Women's Store
0,A,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.00,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.0000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0000,0.00,0.00,0.00,0.000000,0.133333,0.0000,0.000000,0.000000,0.000000,0.0000,0.0000,0.066667,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.0000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.00,0.0000,0.066667,0.0000,0.0000,0.00,0.0000,0.066667,0.0000,0.000000,0.000,0.00,0.00,0.133333,0.200000,0.00,0.0000,0.000,0.0000,0.0000,0.000000,0.000000,0.000000,0.0000,0.0000,0.066667,0.00,0.000000,0.00,0.0000,0.0000
1,B,0.0000,0.010000,0.010000,0.030000,0.020000,0.060000,0.010000,0.000000,0.00,0.01,0.01,0.000000,0.01,0.010000,0.040000,0.01,0.000000,0.02,0.0000,0.01,0.040000,0.02,0.01,0.010000,0.000000,0.0200,0.000000,0.0100,0.010000,0.01,0.020000,0.010000,0.01,0.000000,0.0000,0.01,0.01,0.02,0.000000,0.000000,0.0000,0.000000,0.020000,0.000000,0.0000,0.0000,0.010000,0.010000,0.030000,0.000000,0.000000,0.01,0.010000,0.0000,0.000000,0.0100,0.010000,0.01,0.000000,0.010000,0.01,0.01,0.010000,0.010000,0.010000,0.010000,0.01,0.000000,0.000000,0.01,0.020000,0.000000,0.000000,0.01,0.020000,0.000000,0.010000,0.020000,0.010000,0.03,0.0000,0.050000,0.0000,0.0100,0.01,0.0000,0.000000,0.0000,0.000000,0.000,0.01,0.01,0.010000,0.030000,0.01,0.0000,0.010,0.0000,0.0000,0.000000,0.000000,0.010000,0.0200,0.0000,0.000000,0.00,0.000000,0.01,0.0100,0.0000
2,C,0.0000,0.014706,0.014706,0.073529,0.044118,0.014706,0.014706,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.014706,0.00,0.000000,0.00,0.0000,0.00,0.029412,0.00,0.00,0.029412,0.000000,0.0000,0.014706,0.0000,0.014706,0.00,0.014706,0.014706,0.00,0.000000,0.0000,0.00,0.00,0.00,0.000000,0.029412,0.0000,0.014706,0.044118,0.014706,0.0000,0.0000,0.014706,0.044118,0.014706,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.0000,0.058824,0.00,0.014706,0.014706,0.00,0.00,0.014706,0.014706,0.014706,0.014706,0.00,0.000000,0.000000,0.00,0.014706,0.029412,0.014706,0.00,0.014706,0.000000,0.029412,0.073529,0.000000,0.00,0.0000,0.058824,0.0000,0.0000,0.00,0.0000,0.000000,0.0000,0.000000,0.000,0.00,0.00,0.044118,0.014706,0.00,0.0000,0.000,0.0000,0.0000,0.029412,0.000000,0.044118,0.0000,0.0000,0

#### Confirm the new size

In [29]:
municipio_grouped.shape

(8, 111)

#### Print each municipio along with the top 5 most common venues

In [30]:
num_top_venues = 5

for muni in municipio_grouped['Municipio']:
    print("----"+muni+"----")
    temp = municipio_grouped[municipio_grouped['Municipio'] == muni].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----A----
                venue  freq
0      Soccer Stadium  0.20
1        Soccer Field  0.13
2              Garden  0.13
3  Basketball Stadium  0.07
4       Grocery Store  0.07


----B----
         venue  freq
0          Bar  0.06
1   Restaurant  0.05
2      Brewery  0.04
3  Coffee Shop  0.04
4    BBQ Joint  0.03


----C----
            venue  freq
0     Pizza Place  0.07
1       BBQ Joint  0.07
2  Ice Cream Shop  0.06
3      Restaurant  0.06
4     Supermarket  0.04


----D----
               venue  freq
0        Supermarket  0.10
1          BBQ Joint  0.06
2  Convenience Store  0.06
3        Pizza Place  0.06
4           Pharmacy  0.06


----E----
         venue  freq
0  Pizza Place  0.10
1        Plaza  0.09
2   Restaurant  0.06
3        Beach  0.05
4       Bakery  0.04


----F----
               venue  freq
0        Pizza Place  0.10
1           Pharmacy  0.07
2  Convenience Store  0.07
3       Soccer Field  0.07
4         Restaurant  0.07


----G----
            venue  freq
0     

#### Put it into a dataframe

Function to sort the venues in descending order.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

New dataframe and display the top 10 venues for each neighborhood.

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Municipio']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
municipio_venues_sorted = pd.DataFrame(columns=columns)
municipio_venues_sorted['Municipio'] = municipio_grouped['Municipio']

for ind in np.arange(municipio_grouped.shape[0]):
    municipio_venues_sorted.iloc[ind, 1:] = return_most_common_venues(municipio_grouped.iloc[ind, :], num_top_venues)

municipio_venues_sorted.to_excel('Principales10CategoriasPorMunicipio.xlsx')
municipio_venues_sorted

,Municipio,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A,Soccer Stadium,Soccer Field,Garden,Basketball Court,Restaurant,Shipping Store,Basketball Stadium,Historic Site,Pizza Place,Tennis Court
1,B,Bar,Restaurant,Brewery,Coffee Shop,Pub,BBQ Joint,Soccer Stadium,Gym / Fitness Center,Park,Pizza Place
2,C,Pizza Place,BBQ Joint,Ice Cream Shop,Restaurant,Supermarket,Gym,Soccer Field,Gastropub,Bakery,Nightclub
3,D,Supermarket,BBQ Joint,Convenience Store,Pharmacy,Pizza Place,Plaza,Soccer Stadium,Department Store,Cosmetics Shop,Park
4,E,Pizza Place,Plaza,Restaurant,Beach,Soccer Field,Park,Bakery,Supermarket,Ice Cream Shop,Spanish Restaurant
5,F,Pizza Place,Convenience Store,Grocery Store,Bakery,Supermarket,Pharmacy,Restaurant,Soccer Field,Big Box Store,Ice Cream Shop
6,G,BBQ Joint,Bakery,Garden,Ice Cream Shop,Gym,Gastropub,Soccer Field,Basketball Court,Soccer Stadium,Furniture / Home Store
7,X,Café,Bar,Restaurant,Pizza Place,Hotel,Pub,Coffee Shop,Gym / Fitness Center,Plaza,BBQ Joint


<a id='item4'></a>

## 4. Cluster municipios

Run *k*-means to cluster the municipio into 3 clusters.

In [33]:
# set number of clusters
kclusters = 3

municipio_grouped_clustering = municipio_grouped.drop('Municipio', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(municipio_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 1, 1, 1, 1, 1, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each municipio.

In [34]:
# add clustering labels
municipio_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

municipio_merged = dfm

# merge municipio_grouped with municipio_data to add latitude/longitude for each neighborhood
municipio_merged = municipio_merged.join(municipio_venues_sorted.set_index('Municipio'), on='Municipio')

municipio_merged.head() 

,Municipio,Address,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A,Carlos María Ramírez 862,-34.865730,-56.235827,2,Soccer Stadium,Soccer Field,Garden,Basketball Court,Restaurant,Shipping Store,Basketball Stadium,Historic Site,Pizza Place,Tennis Court
1,B,Joaquín Requena 1701,-34.897304,-56.171140,0,Bar,Restaurant,Brewery,Coffee Shop,Pub,BBQ Joint,Soccer Stadium,Gym / Fitness Center,Park,Pizza Place
2,C,L. A. de Herrera 4547,-34.867148,-56.171454,1,Pizza Place,BBQ Joint,Ice Cream Shop,Restaurant,Supermarket,Gym,Soccer Field,Gastropub,Bakery,Nightclub
3,X,Brito del Pino 1590,-34.899690,-56.156650,0,Café,Bar,Restaurant,Pizza Place,Hotel,Pub,Coffee Shop,Gym / Fitness Center,Plaza,BBQ Joint
4,D,Av. Gral. Flores 4694,-34.845525,-56.154466,1,Supermarket,BBQ Joint,Convenience Store,Pharmacy,Pizza Place,Plaza,Soccer Stadium,Department Store,Cosmetics Shop,Park


Visualize the resulting clusters

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(municipio_merged['Latitude'], municipio_merged['Longitude'], municipio_merged['Municipio'], municipio_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>

Interesting, the cluster are linearly separable which implies a geographical correlation too

## 5. Examine Clusters

#### Cluster 1

In [36]:
municipio_merged.loc[municipio_merged['Cluster Labels'] == 0, municipio_merged.columns[[1] + list(range(5, municipio_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Joaquín Requena 1701,Bar,Restaurant,Brewery,Coffee Shop,Pub,BBQ Joint,Soccer Stadium,Gym / Fitness Center,Park,Pizza Place
3,Brito del Pino 1590,Café,Bar,Restaurant,Pizza Place,Hotel,Pub,Coffee Shop,Gym / Fitness Center,Plaza,BBQ Joint


#### Cluster 2

In [37]:
municipio_merged.loc[municipio_merged['Cluster Labels'] == 1, municipio_merged.columns[[1] + list(range(5, municipio_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,L. A. de Herrera 4547,Pizza Place,BBQ Joint,Ice Cream Shop,Restaurant,Supermarket,Gym,Soccer Field,Gastropub,Bakery,Nightclub
4,Av. Gral. Flores 4694,Supermarket,BBQ Joint,Convenience Store,Pharmacy,Pizza Place,Plaza,Soccer Stadium,Department Store,Cosmetics Shop,Park
5,Estadio Charrúa,Pizza Place,Plaza,Restaurant,Beach,Soccer Field,Park,Bakery,Supermarket,Ice Cream Shop,Spanish Restaurant
6,Av. 8 de Octubre 4700,Pizza Place,Convenience Store,Grocery Store,Bakery,Supermarket,Pharmacy,Restaurant,Soccer Field,Big Box Store,Ice Cream Shop
7,Cno. Castro 730,BBQ Joint,Bakery,Garden,Ice Cream Shop,Gym,Gastropub,Soccer Field,Basketball Court,Soccer Stadium,Furniture / Home Store


#### Cluster 3

In [38]:
municipio_merged.loc[municipio_merged['Cluster Labels'] == 2, municipio_merged.columns[[1] + list(range(5, municipio_merged.shape[1]))]]

,Address,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Carlos María Ramírez 862,Soccer Stadium,Soccer Field,Garden,Basketball Court,Restaurant,Shipping Store,Basketball Stadium,Historic Site,Pizza Place,Tennis Court


The type of venues in the cluster clearly shows Cluster 1 as the most balanced, Cluster 2 shows more open spaces and sport venues, Cluster 3 more places for entertainment

In [39]:
# We'll add a column to identify the clusters by their characteristics to make it easier to analysis at a glance
municipio_merged.insert(5,'Cluster Name','')

In [40]:
municipio_merged.loc[municipio_merged['Cluster Labels']==0,'Cluster Name']='Balanced'
municipio_merged.loc[municipio_merged['Cluster Labels']==1,'Cluster Name']='OpenSpaces'
municipio_merged.loc[municipio_merged['Cluster Labels']==2,'Cluster Name']='Entertainment'

In [41]:
municipio_merged

,Municipio,Address,Latitude,Longitude,Cluster Labels,Cluster Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A,Carlos María Ramírez 862,-34.865730,-56.235827,2,Entertainment,Soccer Stadium,Soccer Field,Garden,Basketball Court,Restaurant,Shipping Store,Basketball Stadium,Historic Site,Pizza Place,Tennis Court
1,B,Joaquín Requena 1701,-34.897304,-56.171140,0,Balanced,Bar,Restaurant,Brewery,Coffee Shop,Pub,BBQ Joint,Soccer Stadium,Gym / Fitness Center,Park,Pizza Place
2,C,L. A. de Herrera 4547,-34.867148,-56.171454,1,OpenSpaces,Pizza Place,BBQ Joint,Ice Cream Shop,Restaurant,Supermarket,Gym,Soccer Field,Gastropub,Bakery,Nightclub
3,X,Brito del Pino 1590,-34.899690,-56.156650,0,Balanced,Café,Bar,Restaurant,Pizza Place,Hotel,Pub,Coffee Shop,Gym / Fitness Center,Plaza,BBQ Joint
4,D,Av. Gral. Flores 4694,-34.845525,-56.154466,1,OpenSpaces,Supermarket,BBQ Joint,Convenience Store,Pharmacy,Pizza Place,Plaza,Soccer Stadium,Department Store,Cosmetics Shop,Park
5,E,Estadio Charrúa,-34.878506,-56.089357,1,OpenSpaces,Pizza Place,Plaza,Restaurant,Beach,Soccer Field,Park,Bakery,Supermarket,Ice Cream Shop,Spanish Restaurant
6,F,Av. 8 de Octubre 4700,-34.858786,-56.133496,1,OpenSpaces,Pizza Place,Convenience Store,Grocery Store,Bakery,Supermarket,Pharmacy,Restaurant,Soccer Field,Big Box Store,Ice Cream Shop
7,G,Cno. Castro 730,-34.859575,-56.213453,1,OpenSpaces,BBQ Joint,Bakery,Garden,Ice Cream Shop,Gym,Gastropub,Soccer Field,Basketball Court,Soccer Stadium,Furniture / Home Store


Finally let's merge with votes to determine satisfaction

In [42]:
municipio_merged = pd.merge(municipio_merged,municipio_votes[['Municipio','FA_Percentage']],on='Municipio', how='left')

In [43]:
municipio_merged

,Municipio,Address,Latitude,Longitude,Cluster Labels,Cluster Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,FA_Percentage
0,A,Carlos María Ramírez 862,-34.865730,-56.235827,2,Entertainment,Soccer Stadium,Soccer Field,Garden,Basketball Court,Restaurant,Shipping Store,Basketball Stadium,Historic Site,Pizza Place,Tennis Court,62.09
1,B,Joaquín Requena 1701,-34.897304,-56.171140,0,Balanced,Bar,Restaurant,Brewery,Coffee Shop,Pub,BBQ Joint,Soccer Stadium,Gym / Fitness Center,Park,Pizza Place,44.98
2,C,L. A. de Herrera 4547,-34.867148,-56.171454,1,OpenSpaces,Pizza Place,BBQ Joint,Ice Cream Shop,Restaurant,Supermarket,Gym,Soccer Field,Gastropub,Bakery,Nightclub,48.49
3,X,Brito del Pino 1590,-34.899690,-56.156650,0,Balanced,Café,Bar,Restaurant,Pizza Place,Hotel,Pub,Coffee Shop,Gym / Fitness Center,Plaza,BBQ Joint,36.28
4,D,Av. Gral. Flores 4694,-34.845525,-56.154466,1,OpenSpaces,Supermarket,BBQ Joint,Convenience Store,Pharmacy,Pizza Place,Plaza,Soccer Stadium,Department Store,Cosmetics Shop,Park,51.02
5,E,Estadio Charrúa,-34.878506,-56.089357,1,OpenSpaces,Pizza Place,Plaza,Restaurant,Beach,Soccer Field,Park,Bakery,Supermarket,Ice Cream Shop,Spanish Restaurant,42.07
6,F,Av. 8 de Octubre 4700,-34.858786,-56.133496,1,OpenSpaces,Pizza Place,Convenience Store,Grocery Store,Bakery,Supermarket,Pharmacy,Restaurant,Soccer Field,Big Box Store,Ice Cream Shop,51.15
7,G,Cno. Castro 730,-34.859575,-56.213453,1,OpenSpaces,BBQ Joint,Bakery,Garden,Ice Cream Shop,Gym,Gastropub,Soccer Field,Basketball Court,Soccer Stadium,Furniture / Home Store,55.37


In [46]:
municipio_summary=municipio_merged[['Municipio','Cluster Name','FA_Percentage']].sort_values(by=['FA_Percentage'])

In [47]:
municipio_summary

,Municipio,Cluster Name,FA_Percentage
3,X,Balanced,36.28
5,E,OpenSpaces,42.07
1,B,Balanced,44.98
2,C,OpenSpaces,48.49
4,D,OpenSpaces,51.02
6,F,OpenSpaces,51.15
7,G,OpenSpaces,55.37
0,A,Entertainment,62.09
